In [9]:
from bs4 import BeautifulSoup

path = 'produced_data/body.xml'

with open(path, 'r', encoding='utf-8') as f:
    def retrieve_person(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        persons = soup.find_all('person')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(person['xml:id'], person.find('persName').text) for person in persons]

        return xml_ids_and_value_labels
    
    def retrieve_places(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        places = soup.find_all('place')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [{place.find('placeName').text: place['xml:id']} for place in places]

        return xml_ids_and_value_labels

# Example usage:

    xml_ids_and_value_labels = retrieve_places(f)

    print(xml_ids_and_value_labels)

[{'Urbino': 'Urbino'}, {'Europa': 'Europe'}, {'Genova': 'Genoa'}, {'Nemours': 'Nemours'}, {'Grecia': 'Greece'}, {'Venezia': 'Venice'}, {'Napoli': 'Naples'}, {'Bologna': 'Bologna'}, {'Museo Civico Medievale di Bologna': 'MCMBologna'}, {'Uffizi': 'Uffizi'}, {'basilica di San Petronio': 'BSanPetronio'}, {'Egitto': 'Egypt'}, {'Palazzo Bolognini Amorini': 'palazzo-salina-amorini'}, {'Italia': 'Italy'}, {'Cremona': 'Cremona'}, {'National Gallery': 'NGLondon'}, {'Roma': 'Rome'}, {'Musei Vaticani': 'VaticanMuseums'}, {'Madrid': 'Madrid'}, {'Museo del Prado': 'PradoMuseum'}, {'Palermo': 'Palermo'}, {'Sicilia': 'Sicily'}, {'Isola di Capri': 'Capri'}, {'monastero della Santissima Annunziata\n': 'Annunziata-Paternò'}, {'Imola': 'Imola'}, {'Parigi': 'Paris'}, {'Francia': 'France'}, {'Pisa': 'Pisa'}, {'Firenze': 'Florence'}, {'Hampton Court': 'HCourt'}, {'chiesa di San Lorenzo in Fonte': 'sanlinfonte'}, {'Accademia nazionale di San Luca': 'accademia-san-luca-roma'}, {'basilica di San Domenico': 'san

In [ ]:
from realNameList import personList
from placeList import placeList
from dictworklocation import worklocationdict
import sparql_dataframe
import pandas
from ratelimit import limits

#store the people missing from wikidata and the multiple entries
missingpeople=[]
multiplepeople=[]

# placeListLabel = list(key for dictionary in placeList for key in dictionary)       
placeListLabel = set(placeList.keys())

@limits(calls=10, period=122)
def personRDF():
       result=[]
       for x,y in personList:        
              
              name = y

              endpoint = 'https://query.wikidata.org/sparql'
              
              queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@it.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "it")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

              df_first = sparql_dataframe.get(endpoint, queryPerson, post='True')
              df = df_first.fillna('')



              #check if there's only one person in wikidata
              
              same = df['human'].unique()
              
              if df.empty:
                     endpoint = 'https://query.wikidata.org/sparql'
              
                     queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@en.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "en")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

                     df_first = sparql_dataframe.get(endpoint, queryPerson, post='True')
                     df = df_first.fillna('')

                     same =df['human'].unique() 

                     if df.empty:

                            missingpeople.append((x,y))
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)


              else:
                     birthPlaceLabel = df['birthPlaceLabel'][0] 
                     deathPlaceLabel = df['deathPlaceLabel'][0]
                     createrdf = """
       <rdf:Description tei:ref='{id}' rdf:about='{wikidataentity}'>
       \t<rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
       \t<rdfs:label xml:lan="it">{label}</rdfs:label>
       \t<crm:P61_has_gender>{genderLabel}</crm:P61_has_gender>
       \t<schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{birthDate}</schema:birthDate>
       \t<schema:deathDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{deathDate}</schema:deathDate>\n""".format(id=x, wikidataentity=df['human'][0], label=df['humanLabel'][0], genderLabel=df['genderLabel'][0], birthDate=df['birthDate'][0], deathDate=df['deathDate'][0])
                     
              #add birth place as referenced in the header or as entity in wikidata
              
                     if birthPlaceLabel in placeList.keys():
                     
                            birthPlaceLabelId= placeList[birthPlaceLabel]

                            createrdf = createrdf + '\t<schema:birthPlace xml:ref="%s">%s</schema:birthPlace>\n'%(birthPlaceLabelId, birthPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:birthPlace rdf:resource="%s">%s<schema:birthPlace>\n'%(df['birthPlace'][0], df['birthPlaceLabel'][0])

              # #add death place as referenced in the header or as entity in wikidata

                     if deathPlaceLabel in placeList.keys():

                            deathPlaceLabelId = placeList[deathPlaceLabel]

              # #ho dei dubbi sul primo caso
                            createrdf = createrdf + '\t<schema:deathPlace xml:ref="%s">%s</schema:deathPlace>\n'%(deathPlaceLabelId, deathPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="%s">%s</schema:deathPlace>\n'%(df['deathPlace'][0], df['deathPlaceLabel'][0])

                     if df['occupation'][0] != '':
                            for occupation in df['occupation'].unique():
                                   occupationLabel = df.loc[df['occupation'] == occupation, 'occupationLabel'].values[0]
                                   createrdf = createrdf + '\t<schema:hasOccupation rdf:resource="%s">%s</schema:hasOccupation>\n'%(occupation, occupationLabel)
                     
                     #if there is at least one workLocation
                     if df['workLocation'][0] != '':
                            print('cè almeno una workLocation')
                            #if the person is an artist
                            if df['humanLabel'][0] in worklocationdict.keys():
                                   print('questa persona è effettivamente unartista')
                                   # for i in worklocationdict[df['humanLabel'][0]]:
                                   #        if i in placeListLabel:

                                   #find all the places named by the book that wikidata consider a workLocation of her
                                   worklocationinwdt = set(df['workLocationLabel'].unique())
                                   worklocationinwdtandbook = worklocationinwdt.intersection(placeListLabel)
                                   print(worklocationinwdtandbook)
                                   for workLocationLabel in list(worklocationinwdtandbook):
                                          # if workLocationLabel in placeList.keys():
                                          workLocationId= placeList[workLocationLabel]
                                          createrdf = createrdf + '\t<schema:workLocation xml:ref="%s">%s</schema:workLocation>\n'%(workLocationId, workLocationLabel)

                                          # else:
                                          #        createrdf = createrdf + '\t<schema:birthPlace rdf:resource="%s">%s<schema:birthPlace>\n'%(df['workLocation'][0], worklocationlabel)


                                          #        worklocationlabel = df.loc[df['workLocationLabel'] == i, 'workLocation'].values[0]
                                          #        createrdf = createrdf + '\t<schema:workLocation rdf:resource"%s">%s</schema:workLocation>\n'%(worklocationlabel, i)
                            
                     
                     
                     
                     createrdf = createrdf + "</rdf:Description>"
                     print(createrdf)
                     result.append(createrdf)
       return result

# print(missingpeople, multiplepeople)
# print(len(missingpeople), len(multiplepeople))
# print(len(result))
# return '\n'.join(result)

# slicer1 = 0
# slicer2 = 1

     
rdf = personRDF()
print(rdf)
print(missingpeople, len(missingpeople))
print(multiplepeople, len(multiplepeople))


In [13]:
#query for places
import sparql_dataframe
from placeList import placeList

def placeRDF(slice1, slice2):
       result = []
       missingplace=[]
       multipleplace=[]
       for i in list(placeList.keys())[slice1:slice2]:

              placelabel = i
              placelabelid = placeList[i]
              

              endpoint = 'https://query.wikidata.org/sparql'

              queryPlace= """SELECT ?place ?placeLabel ?coordinate  WHERE {
              ?place wdt:P31/wdt:P279* wd:Q2221906; # Instances of place (Q2221906) or its subclasses
                     rdfs:label "%s"@it; # Label in English
                     wdt:P625 ?coordinate.
              ?place rdfs:label ?placeLabel. # Get the label
              FILTER(LANG(?placeLabel) = "en") # Filter for English labels
              }
              """%placelabel
              
              df = sparql_dataframe.get(endpoint, queryPlace, post='True')
              
              same = df['place'].unique()

              if df.empty:
                     missingplace.append(i)
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)
              
              else:

                     coordinate = df['coordinate'][0]
                     cleancoordinate = coordinate[6:-1]
                     finalcoordinate = cleancoordinate.split(' ')

                     createrdf ="""
              <rdf:Description tei:ref="{id}" rdf:about="{about}">
                     <rdf:type rdf:resource="http://www.cidoc-crm.org/cidoc-crm/7.1.1/E_53_Place"/>
                     <rdfs:label xml:lang="en">{label}</rdfs:label>
                     <geo:lat>{lat}</geo:lat>
                     <geo:long>{long}</geo:long>
              </rdf:Description>

              """.format(id=placelabelid, wid = df['place'][0], label = placelabel, lat = finalcoordinate[1], long = finalcoordinate[0], about = df['place'][0])
       
                     result.append(createrdf)
       
       
       # print(missingplace, multipleplace)
       return result 

placeRDF(9,10)

['\n              <rdf:Description tei:ref="Uffizi" rdf:about="http://www.wikidata.org/entity/Q51252">\n                     <rdf:type rdf:resource="http://www.cidoc-crm.org/cidoc-crm/7.1.1/E_53_Place"/>\n                     <rdfs:label xml:lang="en">Uffizi</rdfs:label>\n                     <geo:lat>43.768333333</geo:lat>\n                     <geo:long>11.255277777</geo:long>\n              </rdf:Description>\n\n              ']

In [ ]:
from workList import workList
from realNameList import personList
import sparql_dataframe

# def workRDF(slicer1,slicer2):
result = []

for x in list(workList.keys()):
    
    id = 'wd:%s'%x
    
    endpoint = 'https://query.wikidata.org/sparql'
    
    query = """select distinct ?entity ?entityLabel ?author ?authorLabel ?label ?title ?type ?typeLabel ?inception ?creationLocation ?creationLocationLabel ?creationLocationCoordinate ?depicts ?depictsLabel ?movement ?movementLabel ?genre ?genreLabel ?location ?locationLabel 
?locationCoordinate ?city ?cityLabel where{
     BIND (%s AS ?entity)
    ?entity wdt:P170 ?author;
                   rdfs:label ?label;
                   wdt:P31 ?type.
              ?author rdfs:label ?authorLabel.
              ?type rdfs:label ?typeLabel.
              ?entity rdfs:label ?entityLabel
             FILTER(LANG(?authorLabel) = "it")
             FILTER(LANG(?label) = "it")
             FILTER(LANG(?typeLabel) = "it")
             FILTER(LANG(?entityLabel) = "it")
      optional{?entity wdt:P1476 ?title. }
      optional{?entity wdt:P135 ?movement.
               ?movement rdfs:label ?movementLabel.
              FILTER(LANG(?movementLabel) = "it")}
      optional{?entity wdt:P136 ?genre.
               ?genre rdfs:label ?genreLabel.
               FILTER(LANG(?genreLabel) = "it")}           
      optional{?entity wdt:P571 ?inception .}      
      optional{?entity wdt:P180 ?depicts .
               ?depicts rdfs:label ?depictsLabel.
               FILTER(LANG(?depictsLabel) = "it")} 
      optional{?entity wdt:P276 ?location.
               ?location rdfs:label ?locationLabel.
               FILTER(LANG(?locationLabel) = "it")
               optional{?location wdt:P625 ?locationCoordinate.}
               optional{?location wd:P131 ?city.
                       ?city rdfs:label ?cityLabel.
                       FILTER(LANG(?cityLabel) = "it")}}
     optional{?entity wdt:P1071 ?creationLocation.
               ?creationLocation rdfs:label ?creationLocationLabel.
               FILTER(LANG(?creationLocationLabel) = "it")
                optional{?creationLocation wdt:P625 ?creationLocationCoordinate. }}
      }"""%(id)

    df_first = sparql_dataframe.get(endpoint, query, post='True')
    df = df_first.fillna("")

        
    for x,y in personList:
        if df['authorLabel'][0] == y:
            createrdf = '<rdf:Description tei:ref="{ref}" rdf:about="{about}">\n\t<crm:P_108_was_produced_by tei:ref="{ref}">{authorLabel}</crm:P_108_was_produced_by>\n\t<rdfs:label xml:lan="it">{label}</rdfs:label>\n\t<rdf:type rdf:resource="{type}">{typeLabel}</rdf:type>\n\t'.format(ref=x, about=df['entity'][0], authorLabel=y, label = df['entityLabel'][0], type=df['type'][0], typeLabel= df['typeLabel'][0])
        
    if df['inception'][0] != '':
        createrdf = createrdf + '<crm:E_52_Time_Span>%s</crm:E_52_Time_Span>\n\t'%df['inception'][0]
    
    if df['depicts'][0] != '':
        for i in df['depicts'].unique():
            depictsLabel =df.loc[df['depicts'] == i, 'depictsLabel'].values[0]         
            createrdf = createrdf + '<crm:P_138_represents rdf:about="%s">%s</crm:P_138_represents>\n\t'%(i, depictsLabel)
            
        if df['movement'][0] != '':
            for i in df['movement'].unique():
                movementLabel =df.loc[df['movement'] == i, 'movementLabel'].values[0]         
                createrdf = createrdf + '<crm:P_2_has_type rdf:about="%s">%s</crm:P_138_represents>\n\t'%(i, movementLabel)
            
        if df['genre'][0] != '':
            for i in df['genre'].unique():
                genreLabel =df.loc[df['genre'] == i, 'genreLabel'].values[0]         
                createrdf = createrdf + '<crm:P129_is_about rdf:about="%s">%s</crm:P_138_represents>\n\t'%(i, genreLabel)

         
        creationLocationLabel = df['creationLocationLabel'][0]
        
        if df['locationLabel'][0] != '':
            locationLabel = df['locationLabel'][0]
            
            location = df.loc[df['locationLabel'] == locationLabel, 'location'].values[0]  
            #add birth place as referenced in the header or as entity in wikidata
              
            if locationLabel in placeList:
                locationId= placeList[locationLabel]           
                

                createrdf = createrdf + '<schema:location xml:ref="%s">%s</schema:location>\n'%(locationId, locationLabel)
                

            else:
                createrdf = createrdf + '<schema:location rdf:resource="%s">%s<schema:location>\n'%(location, locationLabel)

        
        if df['creationLocationLabel'][0] != '':
            creationLocationLabel = df['creationLocationLabel'][0]
            print(creationLocationLabel)
            ciao = print('ciao')
         
            if creationLocationLabel in placeList.keys():
            
                creationLocationId= placeList[creationLocationLabel]
                creationLocation = df.loc[df['creationLocationLabel'] == creationLocationLabel, 'creationLocation'].values[0]  

                createrdf = createrdf + '<schema:locationCreated xml:ref="%s">%s</schema:locationCreated>\n'%(creationLocationId, creationLocation)

            else:  
                creationLocationId= placeList[creationLocationLabel]
                creationLocation = df.loc[df['creationLocationLabel'] == creationLocationLabel, 'creationLocation'].values[0]  
               
                createrdf = createrdf + '<schema:locationCreated rdf:resource="%s">%s<schema:locationCreated>\n'%(creationLocation, creationLocationLabel)
       
print(createrdf)

In [31]:
from missingPeople import missingPeople
from sparql_dataframe import get

def missing(missingList):
    for x,y in missingList[:20]:
        name = y
        
        endpoint = 'https://query.wikidata.org/sparql'

        query = """SELECT ?human ?humanLabel ?aliases
              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                    skos:altLabel "%s"@en.
                  optional {?human skos:altLabel "Pablo Ruys Picasso"@it}
                     
              ?human rdfs:label ?humanLabel.
                filter(lang(?humanLabel) = "it")
}"""%(name)

        df = get(endpoint, query, post='True')

        print(name)
        print(df)

missing(missingPeople)


Federico di Giorgio Martini
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
 Giuliano de' Medici
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Federico Fregoso
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Lucio Lucinio Crasso
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Lelia
                                      human     humanLabel  aliases
0  http://www.wikidata.org/entity/Q21453257  Lelia Caetani      NaN
Antonio Galeazzo Malvasia
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Conte Guido Pepoli
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Niccolò Tribolo
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Vincenzo Miola
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Asdrubale Anguissola
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Fabrizio Moncada Pignatelli
Empty DataFrame
Columns: [human, humanLabel, aliases]
Index: []
Orazio